# Retrieval-Augmented Generation (RAG) with NVIDIA NIM Endpoints

This notebook demonstrates how to implement a RAG workflow for RFI Responses


## Pull NVIDIA SDKs
Fetches the latest information on NVIDIA SDKs from our NVIDIA Developer website. This information is ultimately used to inform our language model

In [1]:
%run -i scrapeSDKs.py #Takes about a minute

[2025-02-04 21:18:48] Starting scraping process...
[2025-02-04 21:18:50] Filtered content due to 'On-Demand Videos': On-Demand Videos...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'Read More': Read More...
[2025-02-04 21:18:50] Filtered content due to 'On-Demand Videos': On-Demand Videos...
[2025-02-04 21:18:50] Processed 1/115: https://developer.nvidia.com/
[2025-02-04 21:18:50] Processed 2/115: https://developer.nvidia.com/OSMO
[2025-02-04 21:18:51] Filtered content due to 'Resources': Resources...
[2025-02-04 21:18:51] Filtered content due to 'Resources': 

## Import Packages
This code cell imports all of the necessary packages to run the rest of the notebook.

In [1]:
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from IPython.display import display, Markdown
import ipywidgets as widgets
import numpy as np
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

#Stops warning messages from appearing to keep the notebook clean
import warnings
warnings.filterwarnings("ignore")

# Stops warnings and info from the transformers library
from transformers.utils import logging
logging.set_verbosity_error()  

In [11]:
#Relevant directories
nvDIR = "../data/nvidia-text-archive"
partnerDIR = "../data/partner-docs"
rfiDIR = "../data/rfi-docs"

#Create the folders if they do not already exist
for dir in [nvDIR, partnerDIR, rfiDIR]:
    if not os.path.exists(dir):
        print(f'Creating Directory: {dir}')
        os.makedirs(dir)

Creating Directory: ../data/partner-docs


In [12]:
partner_website = "https://www.wwt.com/"

In [13]:
%run -i scrapePartner.py $partner_website $partnerDIR

2025-02-05 18:06:44,646 - INFO - Fetched: https://www.wwt.com/ (1/1)
2025-02-05 18:06:45,236 - INFO - Fetched: https://www.wwt.com/blog/a-year-in-review-world-wide-technology-sustainability-initiatives-2024 (2/37)
2025-02-05 18:06:45,265 - ERROR - Unexpected error processing https://www.wwt.com/blog/a-year-in-review-world-wide-technology-sustainability-initiatives-2024: [Errno 2] No such file or directory: '../data/partner-docs/www.wwt.com/blog/a-year-in-review-world-wide-technology-sustainability-initiatives-2024.txt'
2025-02-05 18:06:45,686 - INFO - Fetched: https://www.wwt.com/article/enterprise-storage-evolution-from-block-to-ai-driven-solutions (3/37)
2025-02-05 18:06:45,718 - ERROR - Unexpected error processing https://www.wwt.com/article/enterprise-storage-evolution-from-block-to-ai-driven-solutions: [Errno 2] No such file or directory: '../data/partner-docs/www.wwt.com/article/enterprise-storage-evolution-from-block-to-ai-driven-solution.txt'
2025-02-05 18:06:46,137 - INFO - Fe

## Initialize API Key
This code cell pulls the NVIDIA_API_Key that was stored as a *Secret* within AI Workbench

In [3]:
nvAPI_Name = "NVIDIA_API_KEY"

if os.environ.get(nvAPI_Name, "").startswith("nvapi-"):
    print(f"Valid {nvAPI_Name} already in environment. Delete to reset")
else:
    nvapi_key = os.environ.get(nvAPI_Name)
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key\nHere are your listed environment variables: {print(os.environ)}"
    os.environ[nvAPI_Name] = nvapi_key

Valid NVIDIA_API_KEY already in environment. Delete to reset


## Check for GPU Availability

If you do not see a GPU available and your host machine has one, **make sure you allocated 1 or more GPUs** in the bottom section called *"Hardware"* of the *"Environment"* tab within the AI Workbench UI. If you need to make this correction, you will need to rebuild your Environment again.

In [4]:
!nvidia-smi --query-gpu=gpu_name --format=csv

name
NVIDIA RTX 3500 Ada Generation Laptop GPU


## Import Helper Functions
This cell imports functions from the [helperFuncs.py](./helperFuncs.py) file 

In [5]:
from helperFuncs import load_documents, addTags, save_uploaded_files, create_callback, rerank_results, rag_query, M2BERTEmbeddings, bokeh_plot, visualize_embeddings

## Document Upload and Processing

Here is where we will create the necessary directories and allow users to upload documents into the appropriate folder with the *File Upload* buttons

In [7]:
# Upload functionality for partner documents
file_upload = widgets.FileUpload(
    accept='.txt, .pdf, .doc, .docx',
    multiple=True,
    description='Partner File Upload',
    layout=widgets.Layout(width='500px')
)

callback = create_callback(partnerDIR)
file_upload.observe(callback, names='value')
display(file_upload)

FileUpload(value=(), accept='.txt, .pdf, .doc, .docx', description='Partner File Upload', layout=Layout(width=…

In [8]:
# Upload functionality for RFI
file_upload = widgets.FileUpload(
    allow='.txt, .pdf, .doc, .docx',
    multiple=True,
    description='RFI File Upload',
    layout=widgets.Layout(width='500px')
)

callback = create_callback(rfiDIR)
file_upload.observe(callback, names='value')
display(file_upload)

FileUpload(value=(), description='RFI File Upload', layout=Layout(width='500px'), multiple=True)

## Chunking & Embedding
Chunking is a significant decision to make in a RAG solution, here are some points to help understand why it is:

**Benefits**
- Improves retrieval precision by breaking down large texts into focused, contextually relevant segments
- Optimizes computational efficiency and response times by reducing per-segment processing load
- Maintains semantic integrity and reduces hallucinations through better context management
- Enables better scalability and dataset maintenance through granular indexing
- ptimizes token usage and associated costs within model context window limits
  
**Challenges**
- Small chunks may lack sufficient context for coherent responses
- Excessive fragmentation can increase computational overhead and API costs
- Large number of small chunks can slow down search performance
- Risk of breaking semantic relationships and natural language flow
- May fragment logically connected information in specialized content (code, technical documentation)

In this example, we use a simple recursive chunking strategy for simplicity-sake, but be aware there are more complex strategies that can take advantage of a given use case like for coding, powerpoints, etc.

In [9]:
os.environ['CHUNK_SIZE'] = '800'
chunk_size = int(os.environ['CHUNK_SIZE']) #If chunk size is too large, it will cause errors downstream depending on the models being leveraged as they have maximum input lengths
chunk_overlap = 100 #More overlap typically gives better context between chunks, but creates more chunks to deal with

In [10]:
documents = load_documents(nvDIR, []) #Load in NVIDIA documents
documents = load_documents(partnerDIR, documents) #Append partner documents
documents = load_documents(rfiDIR, documents) #Append RFI
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

chunked_documents = text_splitter.split_documents(documents)
addTags(chunked_documents)

In [11]:
embeddings = M2BERTEmbeddings(max_seq_length=chunk_size)

%time vectorstore = FAISS.from_documents(chunked_documents, embeddings) #this takes 40-50 seconds on an ADA 3500 GPU

Using device: cuda
-- Bidirectional: True
-- Using Long Conv Residual: True
-- Hyena w: 10
-- Hyena w mod: 1
-- Hyena filter order: 128
-- Hyena filter dropout: 0.2
-- Hyena filter wd: 0.1
-- Hyena filter emb dim: 5
-- Hyena filter lr: 0.001
-- Hyena filter lr pos emb: 1e-05
CPU times: user 28.6 s, sys: 163 ms, total: 28.7 s
Wall time: 28.7 s


In [12]:
visualize_embeddings(vectorstore)

Loading BokehJS ...

In [13]:
context = load_documents(rfiDIR, [])
content = [page.page_content for page in context]

In [14]:
llm = ChatNVIDIA(model="nvdev/nvidia/llama-3.1-nemotron-70b-instruct")

task = llm.invoke(f"Extract the problem statement, goals, constraints, \
and each task in the scope of work in the following: {content}. \
Scope should follow the pattern of <SCOPE TASK TITLE>: <Detail>. Do not summarize. Delimit each section with two empty lines. \
Do not use any markdown or bullets. Expand all acronyms expanded just after each one where found within parentheses if not already explained.")

questions = llm.invoke(f'Please extract all questions asked in the following as a numbered list: {context}. Do not include any headings or markdown. Include all acronyms expanded just after each one where found within parentheses if it has not already been explained.')

if task.response_metadata['finish_reason']=='length':
    print("WARNING: Response artificially truncated due to token length")

In [15]:
display(Markdown(task.content))
print("")
display(Markdown(questions.content))
rfi_details = task.content + questions.content

Here are the extracted information with the specified format:


**Problem Statement**

Underdeveloped requirements can lead to unsatisfactory outcomes in federal contracts, limiting the ability to meet mission needs. The generation of appropriate pre-acquisition documentation has made IT acquisitions increasingly complex, creating challenges for business owners who may not be familiar with the nuance of developing requirements. Ultimately, poorly-written or incomplete pre-acquisition documentation (e.g. Statement of Objectives (SOO), Statement of Work (SOW), Performance Work Statement (PWS), market research, etc.) creates delays in the acquisition lifecycle. Department of State (DoS) program offices with IT acquisition needs require a solution incorporating statutory and regulatory considerations when building requirements documentation.


**Goals**

1. Improved Decision-Making
2. Increased Efficiency and Accuracy
3. Better Contract Outcomes
4. Reduce Costs
5. Compliance in Output


**Constraints**

1. Generative AI tools (e.g. ChatGPT4, Large Language Models (LLM) on huggingface.com) have not been approved for use on Department networks.
2. Time-consuming Information Technology Configuration Control Board (ITCCB) and Authority to Operate (ATO) processes (up to 12 months).
3. Limited expertise among government employees involved in the project (acquisitions SMEs, not AI SMEs).
4. Complete PR packages must be submitted through the agency contract writing system platform for A/OPE/AQM action.
5. Limited DoS IT acquisitions examples suitable for training data.
6. Cloud products require Federal Risk and Authorization Management Program (FedRAMP) authorization.
7. No stakeholder can sponsor the FedRAMP process.
8. Development and implementation of an "acquisition solution" requires contractors with AI engineering expertise and ongoing funding.
9. AI must not perform inherently governmental functions (as described in Federal Acquisition Regulation (FAR) 7.5).
10. Solution must avoid vendor "lock-in" with easily replaceable large language modules.
11. Limited DoS common infrastructure for collaboration and execution with partners.
12. Solution must comply with Executive Order (E.O.) 13960 and 20 Foreign Affairs Manual (FAM) 201.1.
13. Users have varying technical backgrounds.
14. Pending legislation may increase timelines and costs for AI model training.


**Scope of Work**

 

**Identify the Procurement Processes to Automate**
Identify procurement data and processes implementing Artificial Intelligence (AI)/Machine Learning (ML) capabilities to improve efficiency and decision-making. Specify parts of the procurement process requiring human review due to inherently governmental work, compliance with Federal AI policies and frameworks, or high automation costs.


 

**Choose the Right AI Tools**
Select AI tools based on specific procurement processes automated, desired outcomes, and cybersecurity constraints involved with building on DoS OpenNet.


 

**Train the AI Algorithms**
Identify and ascertain ownership of datasets to train the model, incorporating Federal Acquisition Regulations, cybersecurity requirements, DoS-specific contracting requirements, country-specific constraints, exemplar requests for proposals, and other data. Train the algorithm for content, style, structure, and compliance.


 

**Integrate AI with Existing Procurement Systems**
Ensure AI algorithms can access data in relevant systems (e.g., ILMS/ARIBA, System for Award Management (Sam.gov)) and integrate outputs seamlessly back into those systems.


 

**Monitor and Refine**
Monitor output and adjust algorithms as necessary to ensure the solution provides accurate and relevant insights.

Here is the list of extracted questions without headings or markdown, and with acronyms expanded in parentheses where not previously explained:

1. Provide an overview of automated contracting solutions and methodology as you understand them.
2. What are your recommendations for enabling AI (Artificial Intelligence) in a solution?
3. What challenges or barriers should the Government consider in light of the highlighted and other potential constraints?
4. What hurdles has your company faced in obtaining an ATO (Authority to Operate) for a solution that includes large language models?
5. What is the best way to build upon incremental success to bridge the capability gap during prototyping?
6. What FedRAMP (Federal Risk and Authorization Management Program) solutions are you aware of that could meet the definition of done and work within the constraints listed?
7. How much of the process do you think can be automated in the solution (50%? 80%) and why?
8. What criteria should the Government use when evaluating and selecting the solution?
9. What performance or Service Level Agreements (SLAs) are important when evaluating and selecting the solution?
10. Does the AI (Artificial Intelligence) solution allow for independent audits or assessments to ensure compliance, performance, and adherence to standards?
11. What aspects of the process do you think should not be automated by AI (Artificial Intelligence)?
12. How transparent and explainable is the AI (Artificial Intelligence) solution? Can it provide insights into the decision-making process to ensure accountability and fairness?
13. Are there any potential biases or discriminatory outcomes associated with the AI (Artificial Intelligence) solution? How would you address these issues?
14. What considerations should the government be aware of when considering the hidden and long-term costs associated with cloud computing and storage for large language models and scalability?
15. Have you worked on a similar project before? If so, please describe it.

**Follow-up Discussion Questions:**

1. What is your proposed methodology to deliver an automated and AI (Artificial Intelligence)-enabled requirements writing system?
2. Identify and explain how the solution is, or how it could be, AI (Artificial Intelligence)-enabled. Where do you propose AI (Artificial Intelligence) is used in the possible workflows and how is AI (Artificial Intelligence) making decisions/determinations?
3. What taxonomy and ontology do you propose within the AI (Artificial Intelligence) to ensure AI (Artificial Intelligence) output is ethical and using defined terms?
4. Are models pre-trained on existing data, and is that pre-training data tracked? What additional data will be needed to refine the model and the solution candidate?
5. What recommendations do you have for obtaining the data needed?
6. What do you propose are the major milestone outcomes for your proof of concept? What are the estimated timelines for these milestones?
7. What proprietary restrictions does your solution have? Please explain your solution's licensing model (if applicable).
8. How do you accommodate users of varying backgrounds in their interactions with the model?

## Testing the RAG Pipeline

Let's test our RAG pipeline with and without the vector store.

In [41]:
llm = ChatNVIDIA(model="nvdev/meta/llama-3.3-70b-instruct")
partner_name = 'World Wide Technologies (WWT)'

query = f"From the perspective of the company: {partner_name}, Please list specific past experience, products, and certifications to craft a response to the following request for information: {task}. Do not infer something that is not explicitly true and do not generalize"

In [42]:
print("No RAG:\n")
display(Markdown(rag_query(query, llm).content))

No RAG:



As World Wide Technologies (WWT), we have extensive experience in providing innovative solutions to federal agencies. In response to the request for information, we would like to highlight the following specific past experiences, products, and certifications that demonstrate our capabilities in addressing the challenges outlined in the problem statement:

1. **Past Experience:**
   - We have successfully implemented AI/ML solutions for various federal agencies, improving their procurement processes and decision-making efficiency.
   - Our team has expertise in navigating complex regulatory environments, including compliance with Federal Acquisition Regulations (FAR) and cybersecurity requirements.
   - We have experience in training AI algorithms using diverse datasets, ensuring compliance with statutory and regulatory considerations.

2. **Products:**
   - **Procurement Process Automation Tool:** We offer a proprietary tool designed to automate procurement processes using AI/ML capabilities, which can be tailored to meet the specific needs of the Department of State (DoS).
   - **AI-Powered Contract Management System:** Our system integrates with existing procurement systems, such as ILMS/ARIBA and System for Award Management (Sam.gov), to provide seamless management of contracts and compliance with federal regulations.

3. **Certifications:**
   - **FedRAMP Authorized:** Our cloud-based products are FedRAMP authorized, ensuring the security and integrity of data in compliance with federal cloud computing standards.
   - **Compliance with Executive Order (E.O.) 13960 and 20 Foreign Affairs Manual (FAM) 201.1:** We have expertise in ensuring that our solutions comply with these and other relevant federal policies and frameworks.

4. **Addressing Constraints:**
   - **Generative AI Tools:** We are aware of the constraints related to the use of generative AI tools on Department networks and can offer alternative solutions that meet cybersecurity requirements.
   - **ITCCB and ATO Processes:** Our team understands the time-consuming nature of these processes and can provide support to Navigate these efficiently.
   - **Limited Expertise and Data:** We offer training and support to help overcome limited expertise among government employees and can work with available data to train AI models effectively.

5. **Scope of Work:**
   - **Identify Procurement Processes to Automate:** We can assist in identifying procurement data and processes that can be automated using AI/ML, ensuring that inherently governmental functions are preserved.
   - **Choose the Right AI Tools:** Our experts can select appropriate AI tools based on the automated procurement processes, desired outcomes, and cybersecurity constraints.
   - **Train AI Algorithms:** We have the capability to identify, ascertain ownership of, and train AI algorithms using relevant datasets, incorporating all necessary regulatory and contractual requirements.
   - **Integrate AI with Existing Systems:** Our solutions ensure seamless integration with existing procurement systems, facilitating accurate and compliant outputs.
   - **Monitor and Refine:** We provide ongoing support to monitor the output of AI algorithms and refine them as necessary to ensure the solution continues to meet the evolving needs of the DoS.

At WWT, we are committed to providing cutting-edge solutions that address the complex challenges faced by federal agencies. Our experience, products, and certifications position us as a reliable partner for the Department of State in enhancing its procurement processes through the strategic use of AI and ML technologies.

In [43]:
print("RAG with vector store w/o reranking:\n")
response = rag_query(query, llm, vectorstore, rerank=False, show_docs=False, tag_values=['Partner'])
display(Markdown(response.content))

RAG with vector store w/o reranking:



As a leading systems integrator, World Wide Technology (WWT) has extensive experience in providing innovative solutions to the federal government. In response to the request for information, WWT would like to highlight the following past experiences, products, and certifications that demonstrate our capabilities in addressing the challenges outlined in the problem statement:

1. **Procurement Process Automation**: WWT has experience in automating procurement processes for various federal agencies, including the Department of Defense and the Department of Homeland Security. Our team has worked with clients to identify areas where AI/ML capabilities can improve efficiency and decision-making.
2. **AI/ML Solutions**: WWT has developed and implemented AI/ML solutions for federal agencies, including natural language processing, machine learning, and deep learning. Our team has expertise in selecting the right AI tools based on specific procurement processes automated, desired outcomes, and cybersecurity constraints.
3. **Data Integration and Interoperability**: WWT has experience in integrating AI algorithms with existing procurement systems, including ILMS/ARIBA and System for Award Management (Sam.gov). Our team has expertise in ensuring seamless integration of outputs back into these systems.
4. **Cybersecurity and Compliance**: WWT has a strong track record of ensuring cybersecurity and compliance in our solutions, including FedRAMP authorization, ITCCB, and ATO processes. Our team is familiar with the constraints outlined in the problem statement, including the need to avoid vendor "lock-in" and comply with Executive Order (E.O.) 13960 and 20 Foreign Affairs Manual (FAM) 201.1.
5. **Training and Development**: WWT has experience in training AI algorithms for content, style, structure, and compliance. Our team has expertise in identifying and ascertaining ownership of datasets to train the model, incorporating Federal Acquisition Regulations, cybersecurity requirements, and DoS-specific contracting requirements.

Some of the specific products and services that WWT offers include:

1. **Atom AI**: WWT's AI-powered solution for contract analysis and review, which can help improve decision-making and reduce costs.
2. **RFP Assistant**: WWT's AI-powered solution for request for proposal (RFP) development and review, which can help improve efficiency and accuracy.
3. **Data Analytics**: WWT's data analytics services, which can help federal agencies gain insights into their procurement processes and make data-driven decisions.

WWT holds various certifications, including:

1. **ISO 9001:2015**: Quality management system certification
2. **ISO 27001:2013**: Information security management system certification
3. **FedRAMP**: Authorization for cloud products and services

WWT's team of experts includes:

1. **AI/ML Engineers**: Experienced in developing and implementing AI/ML solutions for federal agencies
2. **Procurement Specialists**: Experienced in federal procurement processes and regulations
3. **Cybersecurity Experts**: Experienced in ensuring cybersecurity and compliance in AI/ML solutions

By leveraging our experience, products, and certifications, WWT is well-positioned to address the challenges outlined in the problem statement and provide a comprehensive solution for the federal government.

In [44]:
print("RAG with vector store with reranking:\n")
response, docs = rag_query(query, llm, vectorstore, rerank=True, show_docs=True, tag_values=['Partner'])
display(Markdown(response.content))
print(docs)

RAG with vector store with reranking:



As a leading technology solutions provider, World Wide Technology (WWT) has extensive experience in delivering innovative solutions to federal government agencies. In response to the request for information, WWT would like to highlight the following specific past experience, products, and certifications that align with the requirements:

1. **Past Experience:**
	* WWT has a proven track record of supporting federal government agencies in their IT modernization efforts, including the Department of State (DoS).
	* Our team has expertise in developing and implementing AI/ML solutions, including natural language processing (NLP) and machine learning (ML) algorithms.
	* We have experience in working with various federal agencies, including the DoS, to improve their procurement processes and reduce costs.
2. **Products:**
	* WWT's Atom Ai platform, powered by GPT-4, can be leveraged to automate procurement processes, improve decision-making, and increase efficiency.
	* Our Advanced Technology Center (ATC) provides a collaborative environment for federal agencies to test and evaluate new technologies, including AI/ML solutions.
3. **Certifications:**
	* WWT is a certified FedRAMP Third-Party Assessment Organization (3PAO), which enables us to support federal agencies in achieving FedRAMP compliance for their cloud products.
	* Our team holds various certifications, including ITIL, Six Sigma, and Agile, which demonstrate our commitment to delivering high-quality services and solutions.

To address the constraints and goals outlined in the problem statement, WWT proposes the following approach:

1. **Identify Procurement Processes to Automate:** WWT will work closely with the DoS to identify areas where AI/ML can be applied to improve efficiency and decision-making in the procurement process.
2. **Choose the Right AI Tools:** Our team will select AI tools that align with the specific procurement processes automated, desired outcomes, and cybersecurity constraints involved with building on DoS OpenNet.
3. **Train AI Algorithms:** WWT will identify and ascertain ownership of datasets to train the model, incorporating Federal Acquisition Regulations, cybersecurity requirements, DoS-specific contracting requirements, and other relevant data.
4. **Integrate AI with Existing Procurement Systems:** Our team will ensure that AI algorithms can access data in relevant systems (e.g., ILMS/ARIBA, System for Award Management (Sam.gov)) and integrate outputs seamlessly back into those systems.
5. **Monitor and Refine:** WWT will monitor output and adjust algorithms as necessary to ensure the solution provides accurate and relevant insights.

By leveraging WWT's expertise, products, and certifications, we are confident that we can deliver a solution that meets the goals and constraints outlined in the problem statement, ultimately improving the procurement process for the DoS.

[Document(id='309fce88-f191-447b-b7cd-ff83f03f474e', metadata={'source': '../data/partner-docs/Part 2_ Inside Atom Ai – How Retrieval Mechanisms Shape AI Responses - WWT.pdf', 'page': 8, 'page_label': '9', 'tag': 'Partner'}, page_content='which cannot be accomplished through document retrieval. For Atom, examples of\nthese questions include: \n"How many projects have we done with X client?"\n"What are some projects in which X technology was used?"\n"Who was the project manager on X project?"\nGetting the LLM to consistently generate high-quality SQL queries can be a difficult\ntask given the structured nature of SQL. During our experimentation, we identified a\nfew best practices to make this text-to-SQL module perform as well as possible:\nLLM choice: Since the LLM\'s accuracy and ability to follow instructions are so\nimportant for this application, it is vital to use the best model available. In our\ncase, we chose GPT-4 as our LLM.\nData curation: Don\'t just provide the LLM with a

In [20]:
responses = []
for section in task.content.split('\n\n'):
    if section in [' ', '']:
        continue
    print(section)
    query = f'Please highlight how World Wide Technologies (WWT) might leverage their generative AI expertise enhanced with NVIDIA technologies to address the following section: {section}'
    response = rag_query(query, llm, vectorstore)
    responses.append(response)
    display(Markdown(response))

Here are the extracted information with the specified format:


TypeError: Markdown expects text, not AIMessage(content='It seems like there is no specific section or information mentioned in your query for me to highlight how WWT (World Wide Technologies) might leverage their generative AI expertise enhanced with NVIDIA technologies. However, if you\'re interested, I can provide a general overview of how WWT could potentially utilize such technologies.\n\nWWT is known for its expertise in various technologies, including generative AI. By integrating NVIDIA technologies like those mentioned in the provided sources (e.g., NVIDIA Isaac Sim, TensorRT-optimized engines, NVIDIA Maxine Learning Library), WWT could enhance its offerings in several areas:\n\n1. **Accelerated AI Research and Development**: WWT could leverage NVIDIA\'s AI acceleration platforms (e.g., TensorRT, NVIDIA Maxine) to speed up the development and testing of generative AI models across various domains, including computer vision, natural language processing, and more. This accelerat